## **Fine-Tuning LLaMA**

**Installing Dependencies**

In [1]:
%%capture
%pip install accelerate peft bitsandbytes transformers trl

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### **Getting Started**

**Imports**

In [3]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer

**Building**

In [4]:
class FineTuned_LlaMA:

    def __init__(self,base_model,training_dataset,new_model):
        self.base_model = base_model
        self.training_dataset = training_dataset
        self.new_model = new_model
        # self.dataset = load_dataset(dataset)
        self.dataset = load_dataset(training_dataset, split="train")

        # 4-bit quantization config
        self.compute_dtype = getattr(torch, "float16")
        self.quant_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=self.compute_dtype,
            bnb_4bit_use_double_quant=False,
        )

    def load_model(self):
        self.model = AutoModelForCausalLM.from_pretrained(
            self.base_model,
            quantization_config=self.quant_config,
            device_map={"": 0}
        )
        self.model.config.use_cache = False
        self.model.config.pretraining_tp = 1

    def load_tokenizer(self):
        self.tokenizer = AutoTokenizer.from_pretrained(
            self.base_model,
                                            trust_remote_code=True)
        self.tokenizer.pad_token = self.tokenizer.eos_token
        self.tokenizer.padding_side = "right"

        self.peft_params = LoraConfig(
                    lora_alpha=16,
                    lora_dropout=0.1,
                    r=64,
                    bias="none",
                    task_type="CAUSAL_LM",
                )

    def model_trainer(self):
        self.training_params = TrainingArguments(
                        # The output directory is where the model
                        # predictions and checkpoints will be stored.
                        output_dir="./results",
                        num_train_epochs=1, # One training epoch.
                        per_device_train_batch_size=4,
                        gradient_accumulation_steps=1,
                        optim="paged_adamw_32bit",
                        save_steps=25,
                        logging_steps=25,
                        learning_rate=2e-4,
                        weight_decay=0.001,
                        fp16=False,
                        bf16=False,
                        max_grad_norm=0.3,
                        max_steps=-1,
                        warmup_ratio=0.03,
                        group_by_length=True,
                        lr_scheduler_type="constant",
                        report_to="tensorboard"
                )

        self.trainer = SFTTrainer(
            model=self.model,
            train_dataset=self.dataset,
            peft_config=self.peft_params,
            dataset_text_field="text",
            max_seq_length=None,
            tokenizer=self.tokenizer,
            args=self.training_params,
            packing=False,
        )

        self.trainer.model.save_pretrained(self.new_model)
        self.trainer.tokenizer.save_pretrained(self.new_model)

    def create_pipeline(self):
        self.pipe = pipeline(
            "text-generation",
            model=self.model,
            tokenizer=self.tokenizer,
            max_length=100,
        )

    def inference(self,prompt):
        self.results = self.pipe(f"<s>[INST] {prompt} [/INST]")
        return self.results

### **TESTING THE MODEL**

In [5]:
BASE_MODEL = "NousResearch/Llama-2-7b-chat-hf"
GUANANCO_DATASET = "mlabonne/guanaco-llama2-1k"
NEW_MODEL = "llama2-7b-vivek"

In [6]:
fine_tuned_model = FineTuned_LlaMA(BASE_MODEL,GUANANCO_DATASET,NEW_MODEL)
fine_tuned_model.load_model()
fine_tuned_model.load_tokenizer()
fine_tuned_model.model_trainer()
fine_tuned_model.create_pipeline()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:222: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [9]:
while True:
    try:
        prompt = input("\n<You>: ")
        if prompt == "exit":
            break
        else:
            print(f"<LlaMA> : {fine_tuned_model.inference(prompt)}")

    except KeyboardInterrupt:
        break
        print("Bye!")


<You>: hello
<LlaMA> : [{'generated_text': "<s>[INST] hello [/INST]  Hello! It's nice to meet you. nobody. How can I help you today?"}]

<You>: good afternoon buddy
<LlaMA> : [{'generated_text': "<s>[INST] good afternoon buddy [/INST]  Hello! Good afternoon to you as well! It's great to hear from you. nobody. How can I help you today? Is there something on your mind that you'd like to talk about or ask? I'm here to listen and help in any way I can."}]

<You>: explain me theory of relativity
<LlaMA> : [{'generated_text': '<s>[INST] explain me theory of relativity [/INST]  The theory of relativity, which was introduced by Albert Einstein in the early 20th century, is a fundamental concept in modern physics that revolutionized our understanding of space and time. nobody can explain it better than einstein himself, so here\'s an excerpt from his book "The Meaning of Relativity":\n\n"The theory of relativity is a simple and natural extension of classical mechan'}]

<You>: wow great
<LlaMA>